# Main Statefarm Redux
Here we follow the main statefarm notebook, on our reduxed dataset.

In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
%matplotlib inline
from __future__ import print_function, division
path = "data/state/"
#path = "data/state/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

Using Theano backend.


In [3]:
!pwd
%cd /home/ubuntu/courses/deeplearning1/nbs/kaggle_data/state-farm-redux

/home/ubuntu/courses/deeplearning1/nbs
/home/ubuntu/courses/deeplearning1/nbs/kaggle_data/state-farm-redux


In [4]:
path_to_data = "/home/ubuntu/courses/deeplearning1/nbs/kaggle_data/state-farm-redux/"

In [5]:
batch_size=32

In [6]:
batches = get_batches(path_to_data +'train', batch_size=batch_size)
val_batches = get_batches(path_to_data +'valid', batch_size=batch_size*2, shuffle=False)

Found 16564 images belonging to 10 classes.
Found 5786 images belonging to 10 classes.


In [7]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path_to_data)

Found 16564 images belonging to 10 classes.
Found 5786 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [ ]:
# instead of batching we can also load up all data into an array
trn = get_data(path_to_data +'train')
save_array(path_to_data +'results/trn.dat', trn)
del trn

val = get_data(path_to_data +'valid')
save_array(path_to_data +'results/val.dat', val)
del val

Found 16564 images belonging to 10 classes.


In [10]:
val = load_array(path_to_data +'results/val.dat')
trn = load_array(path_to_data +'results/trn.dat')

MemoryError: 

## Re-run experiments
on the entire set now.

In [8]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

In [9]:
model = conv1(batches)

Epoch 1/2
16564/16564 [==============================] - 418s - loss: 0.2171 - acc: 0.9462 - val_loss: 1.9356 - val_acc: 0.4601
Epoch 2/2
16564/16564 [==============================] - 401s - loss: 0.0136 - acc: 0.9989 - val_loss: 1.9501 - val_acc: 0.5328
Epoch 1/4
16564/16564 [==============================] - 403s - loss: 0.0065 - acc: 0.9992 - val_loss: 1.8539 - val_acc: 0.5328
Epoch 2/4
12672/16564 [=====================>........] - ETA: 75s - loss: 0.0034 - acc: 0.9998

KeyboardInterrupt: 

### Data Augmentation

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path_to_data +'train', gen_t, batch_size=batch_size)

Found 16564 images belonging to 10 classes.


In [ ]:
model = conv1(batches)

Epoch 1/2
16564/16564 [==============================] - 413s - loss: 1.2668 - acc: 0.5934 - val_loss: 1.5601 - val_acc: 0.5149
Epoch 2/2
 8416/16564 [==============>...............] - ETA: 164s - loss: 0.7115 - acc: 0.7779

In [ ]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=15, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

### Three conv + maxpool and dropout
The results from above might be good but unstable - plus we want to try more complex models

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path_to_data+'train', gen_t, batch_size=batch_size)

In [ ]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(128,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

In [ ]:
model.compile(Adam(lr=10e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

In [ ]:
model.optimizer.lr=0.001

In [ ]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

In [ ]:
model.optimizer.lr=0.00001

In [ ]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

### Using vgg16
It's likely that we want to keep all the weights up to the end of convolutional layers, since the same filters might be useful.

In [ ]:
vgg = Vgg16()
model = vgg.model
last_conv_idx = [i for i, l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx + 1]

In [ ]:
conv_model = Sequential(conv_layers)

In [ ]:
# when precomputing features we need to set shuffle to false
batches = get_batches(path_to_data + "train/", batch_size=batch_size, shuffle=False)
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames, test_filenames) = get_classes(path_to_data)

In [ ]:
test_batches = get_batches(path_to_data + "test/", batch_size=batch_size, shuffle=False)
val_batches = get_batches(path_to_data + "valid/", batch_size=batch_size, shuffle=False)

In [ ]:
conv_feat = conv_model.predict_generator(batches, batches.nb_sample)
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)
conv_test_feat = conv_model.predict_generator()

In [ ]:
save_array(path_to_data+'results/conv_val_feat.dat', conv_val_feat)
save_array(path_to_data+'results/conv_test_feat.dat', conv_test_feat)
save_array(path_to_data+'results/conv_feat.dat', conv_feat)

In [ ]:
conv_feat = load_array(path_to_data+'results/conv_feat.dat')
conv_val_feat = load_array(path_to_data+'results/conv_val_feat.dat')
conv_val_feat.shape

#### batchnorm on pretrained conv layers
We have effectively taken the vgg conv layers as they were and used them to precompute features from our train/test/val batches. We need to take those predictions and use them as input to make predictions about our 10 classes.

In [ ]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape = conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(p/2),
        Dense(128, activation="relu"),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation="softmax"),
    ]

In [ ]:
p = 0.8

In [ ]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=1, 
            validation_data=(conv_val_feat, val_labe ls))

In [ ]:
bn_model.optimizer.lr = 0.01

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=1, 
            validation_data=(conv_val_feat, val_labels))

In [ ]:
bm_model.save_weights(path_to_data + "/models/conv_bn.h5")

## Next up - precomputed conv features of augmented data followed by batchnorm

## Pseudolabeling

## Submitting